In [ ]:
EXPERIMENT_NAME = 'Simple notebook pipeline'        # Name of the experiment in the UI
BASE_IMAGE = 'tensorflow/tensorflow:2.0.0b0-py3'

In [ ]:
import kfp
import kfp.dsl as dsl
import kfp.components as comp
from kfp import compiler



```
### This is formatted as code
### Define the storage(minIO, gcs, pvc)
```

# Deal with data input/ouput, model store, and other parameters

# Deal with model code

# Build Components and pipeline (fetch data, training, deploy to seldon, can use tfx, dvc, mlflow, tfboard, mlrun)
# make them python file or in notebook still

In [ ]:
@dsl.python_component(
    name='add_op',
    description='adds two numbers',
    base_image=BASE_IMAGE  # you can define the base image here, or when you build in the next step. 
)
def add(a: float, b: float) -> float:
    '''Calculates sum of two arguments'''
    print(a, '+', b, '=', a + b)
    return a + b

In [ ]:
# Convert the function to a pipeline operation.
add_op = components.func_to_container_op(
    add,
    base_image=BASE_IMAGE, 
)

### f you have a image or want to build in python file

In [ ]:
def random_num_op(low, high):
  """Generate a random number between low and high."""
  return dsl.ContainerOp(
      name='Generate random number',
      image='python:alpine3.6',
      command=['sh', '-c'],
      arguments=['python -c "import random; print(random.randint($0, $1))" | tee $2',
        str(low), str(high), '/tmp/output'],
      file_outputs={'output': '/tmp/output'}
    )


### Build a pipeline

In [ ]:
@dsl.pipeline(
   name='Calculation pipeline',
   description='A toy pipeline that performs arithmetic calculations.'
)
def calc_pipeline(
   a: float =0,
   b: float =7
):
    #Passing pipeline parameter and a constant value as operation arguments
    add_task = add_op(a, 4) #Returns a dsl.ContainerOp class instance. 
    
    #You can create explicit dependency between the tasks using xyz_task.after(abc_task)
    add_2_task = add_op(a, b)
    
    add_3_task = add_op(add_task.output, add_2_task.output)

In [ ]:
@dsl.pipeline(
  name='Conditional execution pipeline',
  description='Shows how to use dsl.Condition().'
)
def flipcoin_pipeline():
  flip = flip_coin_op()
  with dsl.Condition(flip.output == 'heads'):
    random_num_head = random_num_op(0, 9)
    with dsl.Condition(random_num_head.output > 5):
      print_op('heads and %s > 5!' % random_num_head.output)
    with dsl.Condition(random_num_head.output <= 5):
      print_op('heads and %s <= 5!' % random_num_head.output)

  with dsl.Condition(flip.output == 'tails'):
    random_num_tail = random_num_op(10, 19)
    with dsl.Condition(random_num_tail.output > 15):
      print_op('tails and %s > 15!' % random_num_tail.output)
    with dsl.Condition(random_num_tail.output <= 15):
      print_op('tails and %s <= 15!' % random_num_tail.output)

In [ ]:
@dsl.pipeline(
  name='financial time series',
  description='Train Financial Time Series'
)
def preprocess_train_deploy(
        bucket: str = '<bucket>',
        cutoff_year: str = '2010',
        tag: str = '4',
        model: str = 'DeepModel'
):
  """Pipeline to train financial time series model"""
  preprocess_op = Preprocess('preprocess', bucket, cutoff_year).apply(
    gcp.use_gcp_secret('user-gcp-sa'))
  #pylint: disable=unused-variable
  train_op = Train('train', preprocess_op.output, tag,
                   bucket, model).apply(gcp.use_gcp_secret('user-gcp-sa'))
  with dsl.Condition(train_op.outputs['accuracy'] > 0.7):
    deploy_op = Deploy('deploy', tag, bucket).apply(gcp.use_gcp_secret('user-gcp-sa'))


# Compile the pipeline for download or send them 

### Create run directly

In [ ]:
arguments = {'a': '7', 'b': '8'}
# Launch a pipeline run given the pipeline function definition
kfp.Client().create_run_from_pipeline_func(calc_pipeline, arguments=arguments, 
                                           experiment_name=EXPERIMENT_NAME)
# The generated links below lead to the Experiment page and the pipeline run details page, respectively

### Or Compile it then create exp
### https://github.com/kubeflow/examples/blob/master/pipelines/simple-notebook-pipeline/Simple%20Notebook%20Pipeline.ipynb
> Indented block



In [ ]:
# Compile the pipeline
pipeline_func = calc_pipeline
pipeline_filename = pipeline_func.__name__ + '.pipeline.zip'
compiler.Compiler().compile(pipeline_func, pipeline_filename)

In [ ]:
client = kfp.Client()
experiment = client.create_experiment(EXPERIMENT_NAME)

In [ ]:
# Specify pipeline argument values
arguments = {'a': '7', 'b': '8'}

# Submit a pipeline run
run_name = pipeline_func.__name__ + ' run'
run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename, arguments)

# The generated link below leads to the pipeline run information page.